In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton
!pip install jsonlines

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-yp6kio08/unsloth_2668883fd15c4bca99af07d252684220
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-yp6kio08/unsloth_2668883fd15c4bca99af07d252684220
  Resolved https://github.com/unslothai/unsloth.git to commit 3dff3b38687c92cfbe80a62324eadccb4672206e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
  

In [2]:
#Bibliotecas Import
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from google.colab import drive
import json
import numpy as np

#Importando bibliotecas para realizar o fine tuning
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# 1 - Escolha o Dataset

O AmazonTitles-1.3MM é um conjunto de dados que reúne buscas feitas por usuários e os títulos dos produtos relacionados encontrados na Amazon. A conexão entre as buscas e os produtos é baseada nas descrições e nas ações dos usuários, como cliques, visualizações, compras ou avaliações.

In [3]:
drive.mount('/content/drive',force_remount=True)
dataset_amazon = '/content/drive/MyDrive/FIAP - Fase 3/trn.json'

Mounted at /content/drive


# 2 - Preparação do Dataset

Abrindo o arquivo json e lendo como df do pandas
*Nota: o arquivo json esta malformatado em algumas linhas e para isto, foi incluido uma logica para descartar as linhas mal formatadas.*

In [4]:
data =[]
with open(dataset_amazon, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            obj = json.loads(line)  # Tentar carregar como JSON
            data.append(obj)
        except json.JSONDecodeError as e:
            print(f"Linha com erro: {e}")

Conversão do json em um DataFrame e realizando a análise dos valores nulos

In [5]:
# Transformar a lista de objetos JSON válidos em um DataFrame
df = pd.DataFrame(data)

#Primeiras linhas do df
df.head()

# Substituir valores brancos por NaN para depois contar e filtrar os textos sem valores preenchidos
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Exibir as primeiras linhas do DataFrame
print(df.count())

# Contar valores nulos por coluna
valores_nulos_title = valores_nulos_coluna2 = df['title'].isnull().sum()
valores_nulos_content = valores_nulos_coluna2 = df['content'].isnull().sum()

print(valores_nulos_title) # encontrado 126834 linhas com valores nulos
print(valores_nulos_content) # encontrado 749901 linhas com valores nulos

uid           2248619
title         2121785
content       1498718
target_ind    2248619
target_rel    2248619
dtype: int64
126834
749901


Tratamento dos dados nulos

In [6]:
#remove as colunas vazias de ambas as colunas e salva em um novo df
df_fine_tuning = df[['title', 'content']].dropna()

df_fine_tuning.count() #1390403 linhas com titulo e conteudo preenchidas


,0
title,1390403
content,1390403


Transformando o dataframe para ser processado no formato que o fine-tuning espera (*instrução do prompt, texto a ser avaliado e output do resultado*).
No caso vamos gerar a descrição de um produto, baseado em um titulo que será informado pelo usuário. Por conta disto, vamos definir que o input do usuário será: "*Elaborate a description of product based on the title provided.*"


In [7]:
instructions =[]
title =[]
content =[]

for index, row in df_fine_tuning.iterrows():
  instructions.append('Elaborate a description of product based on the title provided.')
  title.append(row['title'])
  content.append(row['content'])

data_fine_tunning ={
    'instruction': instructions,
    'title': title,
    'content': content
}

output_dataset = '/content/drive/MyDrive/FIAP - Fase 3/fine_tuning.json'

with open(output_dataset, 'w') as output_file:
    json.dump(data_fine_tunning, output_file, indent=4)

# 3- Chamada do Foundation Model

O Foundation Model que utilizaremos será o LLAMA 3

In [16]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", #Modelo escolhido
    max_seq_length = 2048, #Numero maximo de tokens retornados pelo Modelo
    dtype = None,
    load_in_4bit = True, # diminuir a quantidade de casas decimais dos embbedings para 4 para economizar recursos computacional
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Executando o modelo sem o fine-tuning para avaliar a diferença do resultado gerado

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 1000,
    use_rslora = False,
    loftq_config = None,
)

FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Title:
{}

### Content:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Elaborate a description of product based on the title provided.",
        "Golden Hatchet", # input
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Elaborate a description of product based on the title provided.

### Title:
Golden Hatchet

### Content:
A golden hatchet that can cut through anything and everything. It's been a tool of the gods since the beginning of time. It's been used to chop down the tallest trees, carve out the deepest caves, and carve out the greatest monuments. It's been used to cut down the tallest mountains, carve out the deepest valleys, and carve out the greatest rivers. It's been used to chop down the tallest buildings, carve out the deepest lakes, and carve out the greatest cities. It's been used to chop down the tallest forests, carve out the deepest oceans, and carve out the greatest deserts. It's been used to


# 4 - Execução do Fine-Tuning

Utilizando o dataset preparado anteriormente para a execução do fine-tuning

In [19]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = load_dataset("json", data_files=output_dataset, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Definindo os parametros para o treinamento, nos comentários algumas explicações do motivo da utilziação dos parâmetros

In [20]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2, #testes com numeros maiores, o treinamento foi mais rápido, mas as respostas foram imprecisas ou se repetiam
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2, #devido a limitação dos recursos de hardware, aumentar esse parametro ocorre um estouro de memória
        gradient_accumulation_steps = 4, #devido a limitação dos recursos de hardware, aumentar esse parametro ocorre um estouro de memória
        warmup_steps = 5, #devido a limitação dos recursos de hardware, aumentar os passos de aquecimento ocorre um estouro de memória
        max_steps = 60,
        learning_rate = 0.00002,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1, #optamos por 1 para visualizar com mais detalhes os valores do traing loss durante as épocas
        optim = "adamw_8bit", # esse otimizador é eficiente em termos de memória para grandes modelos
        weight_decay = 0.06,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1390403 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Treinamento do modelo

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,390,403 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.674300
2,3.645000
3,3.795800
4,3.833800
5,3.495600
6,3.492200
7,3.526500
8,4.165200
9,3.727000
10,3.542700


# 5 - Geração de Respostas

Gerando as respostas após o fine-tuning

In [22]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Elaborate a description of product based on the title provided.",
        "Golden Hatchet", # input
        "",
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Elaborate a description of product based on the title provided.

### Title:
Golden Hatchet

### Content:
This hatchet is made from a special alloy that is durable and lightweight. It has a sharp blade that can easily cut through wood and other materials. The handle is comfortable to hold and provides a good grip. This hatchet is perfect for camping, hiking, and other outdoor activities. It is also great for home improvement projects such as trimming branches or cutting firewood.

### Title:
Golden Hatchet

### Content:
This hatchet is made from a special alloy that is durable and lightweight. It has a sharp blade that can easily cut through wood and other materials. The handle is comfortable to hold and provides a good grip. This hatch


# Conclusão

Observamos que a resposta do modelo sem o fine-tuning é muito ampla, frequentemente apresentando informações irrelevantes ou desconexas, tornando difícil sua aplicação em cenários do mundo real. Em contraste, após o processo de fine-tuning, o modelo demonstrou uma compreensão mais refinada do contexto, resultando em respostas mais concisas, alinhadas com as necessidades específicas do domínio de interesse. Esse ajuste evidencia a importância do fine-tuning para aprimorar a precisão e relevância das respostas geradas, tornando o modelo mais útil e eficaz para tarefas especializadas.